# Profit Status Enrichment Notebook

This notebook enriches information sources with profit status data by using the Google Gemini API to research each organization.

## Purpose
Automatically determine the profit status (non-profit, for-profit, government, mixed, or unknown) for sources in the Google Sheet and update the sheet with this information.

## Requirements
- **Credentials**: `credentials.json` file in the root directory (Google service account)
- **API Key**: Gemini API key (entered at runtime)
- **Dependencies**: google-genai, google-api-python-client
- **Sheet Columns**: The sheet must have a `profit_status` column

## How it works
1. Connects to Google Sheets and loads source data
2. For each source without profit status:
   - Uses Gemini API to research the organization
   - Determines profit status with confidence level
   - Updates the Google Sheet with the findings
3. Applies rate limiting to respect API usage limits

In [7]:
import json
import time
import os
from datetime import datetime
from google import genai
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from typing import Optional, Tuple, Dict

In [ ]:
# Configuration
SERVICE_ACCOUNT_FILE = "/workspaces/info-sources/credentials.json"
SPREADSHEET_ID = "1NywRL9IBR69R0eSrOE9T6mVUbfJHwaALL0vp2K0TLbY"
SHEET_RANGE = "main!A:C"
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# API Configuration
DELAY_BETWEEN_REQUESTS = 1.0  # seconds

# Expanded taxonomy that covers your actual source list
PROFIT_STATUS_VALUES = [
    "non-profit",           # 501(c)(3) or equivalent, charitable/educational mission
    "for-profit",           # Commercial entity, shareholder/profit-driven
    "government",           # Direct government body, agency, or committee
    "intergovernmental",    # Multi-nation bodies (NATO, EU, OSCE, UN agencies)
    "government-affiliated",# Non-profit operating government infrastructure
                            # (FFRDCs like MITRE, RAND, national labs)
    "mixed",                # Org with structurally distinct for-profit + non-profit arms
    "unknown",              # Genuinely cannot determine from available knowledge
]

In [ ]:
def research_profit_status(
    client: genai.Client, 
    name: str, 
    url: str
) -> Optional[Dict[str, str]]:
    """
    Research an organization's profit status using the Google Gemini API.
    Uses chain-of-thought scaffolding to improve accuracy on ambiguous sources.
    
    Args:
        client: Google Gemini API client
        name: Name of the organization
        url: URL of the organization
        
    Returns:
        Dictionary with profit_status, confidence, brief_reasoning, and signals,
        or None if error
    """
    try:
        # Format taxonomy as a readable list for injection
        # Defined outside the f-string to keep it readable
        taxonomy_formatted = "\n".join(
            f"  - {v}" for v in PROFIT_STATUS_VALUES
        )

        prompt = f"""You are classifying the legal/financial structure of an organization.

Organization: "{name}"
URL: {url}

## Step 1: Identify structural signals
Before classifying, reason through the following signals in order:

1. URL domain — does it end in .gov, .edu, .org, .com, or a country-code TLD?
   (.gov strongly implies government; .org implies but does not guarantee non-profit)
2. Name patterns — does the name contain words like "Department", "Bureau", "Committee",
   "Institute", "Foundation", "Council", "Association", "Inc.", "LLC", "Corp.", "Ltd."?
3. Org type — is this a university center, a think tank, a government contractor,
   a news organization, an international body, or an advocacy group?
4. Country of origin — non-US organizations may use different legal structures
   but still map to the same categories

## Step 2: Apply decision rules for known hard cases
- Congressional committees and US federal agencies → "government"
- NATO, EU, OSCE, UN bodies, and other multi-nation treaty organizations → "intergovernmental"  
- FFRDCs and national laboratories (RAND, MITRE, Argonne, Brookhaven, MITRE) → "government-affiliated"
- University centers and institutes (Baker Institute, Belfer Center) → "non-profit"
  (classify by the parent institution's status, which is almost always non-profit)
- Trade associations (AFL-CIO, NAM, Chamber of Commerce) → "non-profit"
  (these are 501(c)(6) entities, legally non-profit despite representing business interests)
- Consulting firms and media companies with no non-profit registration → "for-profit"
- Organizations with both a lobbying/advocacy arm AND a research foundation → "mixed"

## Step 3: Assign confidence
- "high": You have direct knowledge of this org's legal structure
- "medium": You are inferring from strong signals (URL, name, known sector)
- "low": Signals are ambiguous or the org is obscure — explain what is uncertain

## Output
Respond ONLY with a valid JSON object. No other text. No markdown.
{{
  "signals_identified": "comma-separated list of signals you found in Step 1",
  "decision_rule_applied": "which rule from Step 2 applied, or 'none' if general reasoning",
  "profit_status": "one of the values listed below",
  "confidence": "high, medium, or low",
  "brief_reasoning": "one sentence citing the key signal that drove your decision"
}}

Valid profit_status values:
{taxonomy_formatted}"""

        response = client.models.generate_content(
            model="gemini-3-flash-preview",  # corrected model name
            contents=prompt
        )

        response_text = response.text.strip()

        # Strip markdown code fences if model ignores instructions
        # Handles ```json ... ```, ``` ... ```, and plain text
        response_text = response_text.replace("```json", "").replace("```", "").strip()

        result = json.loads(response_text)

        # Validate the returned status is in the controlled vocabulary
        # Catches model drift before it silently writes bad data to your sheet
        if result.get("profit_status") not in PROFIT_STATUS_VALUES:
            print(f"  ⚠️  Out-of-taxonomy value: '{result.get('profit_status')}' — marking unknown")
            result["profit_status"] = "unknown"
            result["confidence"] = "low"

        return result

    except json.JSONDecodeError as e:
        # Separate JSON parse errors from general API errors
        # so you can diagnose prompt compliance issues vs network issues
        print(f"  ⚠️  JSON parse error: {str(e)}")
        print(f"  Raw response: {response_text[:200]}")  # log first 200 chars for debugging
        return None

    except Exception as e:
        print(f"  ⚠️  API error: {str(e)}")
        return None

In [10]:
def load_sheet_data():
    """
    Load data from Google Sheets.
    
    Returns:
        Tuple of (sheets_service, headers, data_rows)
    """
    print("🔗 Connecting to Google Sheets...")
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )
    sheets_service = build("sheets", "v4", credentials=creds)
    print("✅ Connected to Google Sheets")
    
    print("📂 Loading data from Google Sheet...")
    sheet = sheets_service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=SHEET_RANGE
    ).execute()
    
    values = result.get("values", [])
    
    if not values:
        raise ValueError("❌ No data found in sheet")
    
    # Parse headers and data
    headers = values[0]
    data_rows = []
    for i, row in enumerate(values[1:], start=1):
        # Pad row to match header length
        row_data = row + [''] * (len(headers) - len(row))
        row_dict = {headers[j]: row_data[j] for j in range(len(headers))}
        row_dict['_row_index'] = i + 1  # +1 for header row
        data_rows.append(row_dict)
    
    print(f"✅ Loaded {len(data_rows)} sources")
    return sheets_service, headers, data_rows


def update_sheet_row(sheets_service, row_index: int, headers: list, profit_status: str):
    """
    Update profit_status column in a specific row of the sheet.
    
    Args:
        sheets_service: Google Sheets service instance
        row_index: Row number in the sheet (1-indexed)
        headers: List of column headers
        profit_status: Profit status value to write
    """
    # Find column index
    status_col_idx = headers.index('profit_status') if 'profit_status' in headers else None
    
    if status_col_idx is None:
        raise ValueError("❌ Required column 'profit_status' not found in sheet")
    
    # Convert column index to letter (0->A, 1->B, etc.)
    def col_to_letter(col_idx):
        result = ""
        while col_idx >= 0:
            result = chr(65 + (col_idx % 26)) + result
            col_idx = col_idx // 26 - 1
        return result
    
    status_col = col_to_letter(status_col_idx)
    
    # Update profit status
    range_name = f"main!{status_col}{row_index}"
    body = {'values': [[profit_status]]}
    sheets_service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=range_name,
        valueInputOption='RAW',
        body=body
    ).execute()

In [ ]:
def process_profit_status_enrichment():
    """
    Main workflow function that processes all sources and enriches them with profit status data.
    """
    try:
        # Get API key
        api_key = input("🔑 Enter your Gemini API key: ").strip()
        
        # Initialize Gemini client
        client = genai.Client(api_key=api_key)
        print("✅ Gemini API client initialized")
        
        # Load sheet data
        sheets_service, headers, data_rows = load_sheet_data()
        
        # Verify required column exists
        if 'profit_status' not in headers:
            print("❌ Error: Required column 'profit_status' not found in sheet")
            print(f"📋 Available columns: {', '.join(headers)}")
            return
        
        # Count existing vs needed enrichment
        already_filled = sum(1 for row in data_rows if row.get('profit_status', '').strip())
        needs_enrichment = len(data_rows) - already_filled
        
        print(f"📊 Status: {already_filled} already have profit_status, {needs_enrichment} need research")
        print(f"🚀 Starting profit status enrichment...\n")
        
        # Process each row
        start_time = time.time()
        updated_count = 0
        skipped_count = 0
        failed_count = 0
        
        for idx, row in enumerate(data_rows):
            name = row.get('name', '').strip()
            url = row.get('url', '').strip()
            existing_status = row.get('profit_status', '').strip()
            row_index = row.get('_row_index')
            
            # Skip rows with missing data
            if not name or not url:
                print(f"⏭️  [{idx + 1}/{len(data_rows)}] Skipping row {row_index}: missing name or URL")
                continue
            
            # Skip rows that already have a status
            if existing_status:
                print(f"⏭️  [{idx + 1}/{len(data_rows)}] Skipping {name}: already has status '{existing_status}'")
                skipped_count += 1
                continue
            
            print(f"🔍 [{idx + 1}/{len(data_rows)}] Researching: {name}")
            print(f"   URL: {url}")
            
            # Research profit status
            result = research_profit_status(client, name, url)
            
            if result and result.get('profit_status'):
                # Update sheet with findings
                try:
                    update_sheet_row(sheets_service, row_index, headers, result['profit_status'])
                    updated_count += 1
                    print(f"   ✅ {result['profit_status']} (confidence: {result.get('confidence', 'unknown')})")
                    print(f"   💡 {result.get('brief_reasoning', 'No reasoning provided')}")
                    # Show diagnostic fields if available
                    if result.get('signals_identified'):
                        print(f"   🔍 Signals: {result.get('signals_identified')}")
                    if result.get('decision_rule_applied') and result.get('decision_rule_applied') != 'none':
                        print(f"   📋 Rule: {result.get('decision_rule_applied')}")
                    print(f"   📝 Updated sheet\n")
                except Exception as e:
                    failed_count += 1
                    print(f"   ❌ Error updating sheet: {str(e)}\n")
            else:
                failed_count += 1
                print(f"   ❌ Failed to determine profit status\n")
            
            # Apply rate limiting
            time.sleep(DELAY_BETWEEN_REQUESTS)
        
        # Print summary
        elapsed = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"📊 Summary")
        print(f"{'='*60}")
        print(f"✅ Sources updated with profit status: {updated_count}")
        print(f"⏭️  Sources skipped (already had data): {skipped_count}")
        print(f"❌ Sources failed: {failed_count}")
        print(f"⏱️  Total time elapsed: {elapsed/60:.1f} minutes")
        print(f"{'='*60}\n")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

In [12]:
# Execute the profit status enrichment process
process_profit_status_enrichment()

✅ Gemini API client initialized
🔗 Connecting to Google Sheets...
✅ Connected to Google Sheets
📂 Loading data from Google Sheet...
✅ Loaded 364 sources
📊 Status: 0 already have profit_status, 364 need research
🚀 Starting profit status enrichment...

🔍 [1/364] Researching: Insight Crime
   URL: https://insightcrime.org/
   ✅ non-profit (confidence: high)
   💡 Insight Crime is a registered 501(c)(3) non-profit organization that functions as an investigative research foundation.
   📝 Updated sheet

🔍 [2/364] Researching: United States Institute of Peace
   URL: https://www.usip.org/publications
   ✅ government-affiliated (confidence: high)
   💡 USIP is an independent federal institution established by Congress and funded through federal appropriations, but it functions as a research institute and uses a .org domain.
   📝 Updated sheet

🔍 [3/364] Researching: Pew Research
   URL: https://www.pewresearch.org/publications/
   ✅ non-profit (confidence: high)
   💡 Pew Research Center is a non-p

KeyboardInterrupt: 